# Unit 7: Advanced Data Visualization
-------------------------------------

In Unit 3, we explored how the `matplotlib` package can be used to create, customize and save professional-looking data visualizations. By using Python code to plot our data, we can easily generate plots that have a consistent look for presentations and reports. Additionally, by drawing on other Python packages, such as `pandas`, we can chain together data analysis and plotting steps to create an automated pipeline.

Building on this base, we will introduce two additional plotting topics in this unit. First, we will explain the process for creating faceted plots (where data is split between multiple axes in the same figure). The can be helpful to avoid overcrowding the data into a single plot, or to present data that shares one axis scale, but not the second. For instance, we may have multiple variables in time series data that have different $y$ scales. By plotting these on separate, one above the other, we can share the time ($x$) scale, while providing separate y-tick labels. We'll see an example of this below.

Second, we'll explore the power of the `seaborn` package. This project was built on top of `matplotlib`, and reduces the amount of code that you need to write. This package is specifically designed to streamline the use of data stored in a `pandas.DataFrame`, and will help you out by automatically labeling the $x$ and $y$ axes with the column name and allowing you to easily use a categorical variable to create multiple labeled series in your plot. There won't be as much theory or as many steps in this section - the goal is to provide examples of what is possible in `seaborn`. 

**After completing this unit, you should be able to:**

- Create faceted plots in `matplotlib`
- Use the `seaborn` package for more advanced plots

## 7.1. Creating faceted plots

In Unit 4, we created an example where we plotted a time-series data set, and it's derivative. Let's dissect this code to understand how the subplots were created. The key difference, compared to what we have done previously, it that we have introduced new parameters for the [`plt.subplots()`](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.subplots) function. 

| Parameter | Default Value | Possible Values | Description |
|-----------|---------------|-----------------|-------------|
| `nrows` | 1 | integer | number of rows of plots to create |
| `ncols` | 1 | integer | number of columns of plots to create |
| `sharex` | `False` | `True`, `False`, `'row'`, `'col'` | should each axis share the same x-ticks? |
| `sharey` | `False` | `True`, `False`, `'row'`, `'col'` | should each axis share the same y-ticks? |

In this example, we have created 2 *rows* of plots, with the default value of 1 *column*. Because these share the same time scale, it is appropriate for them to share the same x-tick positions and labels. However, the y-scale for the signal and its derivative are not in the same units, so these are not shared (because the default argument is `False`).

When `nrows` or `ncols` is greater than 1 the `ax` object becomes an array, instead of a single `axis`. So, to select the specific axis, array indexing is required. By default, when `nrows==ncols==1`, the `plt.subplots()` function collapses the array to a scalar value so that indexing is not required. This is why we have not needed to use array indexing previously. In contrast if both `nrows` *and* `ncols` are greater than 1, then the return `ax` array will be 2-dimensional. When this is true, then the axis will be selected in a $row \times column$ form, just like other 2-dimensional matrices. For example, the `axis` in the first row and first column would be selected with `ax[0, 0]`. Try changing this plot to display the two signals side-by-side, instead of top and bottom.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# read a .csv file into a DataFrame variable named df
fid_df = pd.read_csv('../../data/pcr-polyethylene_gc-fid.csv')
fid_df['seconds'] = fid_df['minutes'] * 60

# calculate the first derivative (change in FID signal) / (change in time)
fid_df['1st_derivative'] = fid_df['fid'].diff() / fid_df['seconds'].diff()

# plot the FID signal and its 1st derivative
# the \n in the label is the new-line character to split the label into 2 lines
fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(fid_df['seconds'], fid_df['fid'])
ax[0].set_ylabel('Original\nFID Signal')

ax[1].plot(fid_df['seconds'], fid_df['1st_derivative'])
ax[1].set_xlabel('Seconds')
ax[1].set_ylabel('1st Derivative\nof FID Signal')

Creating faceted plots can be especially useful when we have data that has multiple scales, or if we need to separate data. Separating the plots can be easier for a reader to interpret, versus plotting using multiple y-scales on the same plot.

## 7.2. Plotting using the `seaborn` package

The [`seaborn`](https://seaborn.pydata.org/) package was developed on top of `matplotlib` to provide easy-to-use functions to produce more complicated plots, especially when you there are multiple categories to display on the same plot. Most of the `seaborn` functions are built to integrate with a `pandas.DataFrame` directly, and we can supply the column names, instead of arrays, for the $x$ and $y$ coordinates. I recommend following the link and exploring the gallery to see the different options, and review the code there. 

We'll see that `seaborn` allows us to create complex visuals with just a few simple lines of code. But, there is a trade-off. Because of the work that `seaborn` is doing on your behalf, it can sometimes be more complicated to control the fine details of a plot. For exploratory data analysis, this can be a good trade - you will see progress more quickly. When I have a specific visual that I want to automate, I will typically code that in `matplotlib` directly. One challenge that I have with this package is there is some inconsistency in how the different plot types are coded, and what type of object is returned by the function. This makes it harder to guess how to use a new plot type in `seaborn` and means that I'm more likely to need to refer back to the documentation. 

### 7.2.1. Grouped bar chart

The bar chart is often used to present average values for a set of data. One common enhancement of the basic bar chart is the use of multiple (often colored) bars to represent an additional categorical variable. The [`seaborn.catplot()`](https://seaborn.pydata.org/generated/seaborn.catplot.html) function will execute the pivot table step to average the data and plots the `DataFrame` columns that we have specified by name. Note that we still need to apply the filter, as shown, to select only the tensile modulus data for this plot.

This function returns a seaborn-specific object called a `FacetGrid`, which wraps together the data and plot objects. We can store this as a variable, labeled in the `seaborn` examples as `g`, and use this to further customize the plot. 

A multi-series bar chart like this could be created with what we have learned about `matplotlib` in Unit 3 and `pandas` pivot tables in Unit 4. It would be a good exercise to recreate this plot without the `seaborn` package, and to compare the amount of Python code that is required in the two approaches.

In [ ]:
import seaborn as sns

# read in the physical properties dataset that we have used previously
films_df = pd.read_excel('../../data/film_testing.xlsx', sheet_name='physical_properties')

# filter the data to select just the Elongation at Break property
elongation_df = films_df[films_df['Property']=='Elongation at Break']

# plot the bar chart, with film orientation as a categorical value
# the catplot function returns a seaborn.FacetGrid object
g = sns.catplot(
    data=elongation_df, kind="bar",
    x='FilmID', y="Measurement", hue="Direction", 
    ci="sd", legend_out=False
)

# change the axis labels from the DataFrame column name to user-specified text
g.set_axis_labels("Film Grade", "Elongation at Break (%)")

# update the title of the legend
g.legend.set_title("Film Orientation")

### 7.2.2. Scatter plot with mulitple categories

In Unit 6, we used measured film properties to predict whether a film was BOPET or BOPP. In this unit, we used the `c` parameter of the `Axes.scatter()` function to control the color of the individual points. This indicated which class the point belonged to. One challenge with this approach is that the legend cannot be automatically created to label what these colors represent. This can be done in `matplotlib`, but would require some extra code to plot the data from the two classes using separate calls to `Axes.scatter()`. 

As we saw in the previous example, `seaborn` offers easy-to-use methods for working with categorical data that require very little code. First, we create a new column in our `DataFrame` to represent the $+1/-1$ classification in a text form. Then, we supply this `DataFrame` to the [`seaborn.scatterplot()`](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) function. Here, we observe one of the inconsistencies in the return values of the `seaborn` plotting function. Whereas the previous `catplot()` returned the `FacetGrid` object, the `scatterplot()` returns a `matplotlib.Axes` object. This can be found by reading the documentation for the function. The good news is that we know how to work with the `matplotlib.Axes` already from Unit 3. 

In [ ]:
# film classification dataset from Unit 6
film_class_df = pd.read_csv('../../data/film_classification.csv')

# create a new column in the dataframe to name the 1=>BOPET, -1=>BOPP
film_class_df['FilmType'] = film_class_df['Class'].apply(lambda x: 'BOPET' if x==1 else 'BOPP')

# draw a scatter plot while assigning point colors based on category
# size of the points can also be modified in the scatterplot
ax = sns.scatterplot(x="Dart Impact", y="Tensile Modulus",
                     hue="FilmType", 
                     linewidth=0, palette='tab10',   
                     data=film_class_df)

# any of the Axes functions we learned in Unit 3 can be applied to this Axes object
ax.set_xlabel("Dart Impact Energy (J)")

### 7.2.3. Joint plot

Building on our classification dataset, we may want to visualize the distribution of values on two dimensions. One of the more complex plots that we can create is the [`seaborn.jointplot()`](https://seaborn.pydata.org/generated/seaborn.jointplot.html). This one would be more complicated to code using pure `matplotlib`. 

In this example, we present the same scatter plot of modulus and dart impact values. The joint plot adds the [*kernel density estimation*](https://seaborn.pydata.org/tutorial/distributions.html#tutorial-kde) for each dimension. More information can be found by following the link, but you can think about this as a smoothed histogram. 

In [ ]:
g = sns.jointplot(data=film_class_df, kind="scatter", 
                  x="Dart Impact", y="Tensile Modulus", 
                  hue='FilmType')

Note that there are different allowable values for the `kind` parameter that will produce different styles of plot. In the example below, we select the `reg` king, which will add a linear regression to the scatter data. However, this plot type cannot be used together with the `hue` parameter and multiple data series. So, this looks at the dataset overall. With a single series, both the actual histogram (the bars) and the kernel density estimatation (curves) are provided in each dimension.

In [ ]:
g = sns.jointplot(data=film_class_df, kind="reg",
                  x="Dart Impact", y="Tensile Modulus")

### 7.2.4. Linear regression plot

In Unit 6, we learned to conduct regression analysis using Python. In some cases, it may be helpful to quickly visualize the best-fit line using [`seaborn.lmplot()`](http://seaborn.pydata.org/generated/seaborn.lmplot.html). As with other `seaborn` plotting functions, we can supply data in a `DataFrame` and split the results by a categorical value. In addition to the `hue` parameter for categories, we can supply the additional `row` and `col` parameters to build a faceted grid with additional categorical variables.

Using `seaborn` is not as flexible as conducting the regression analysis separately in *scikit-learn*, and does not provide direct access to the model that is used to create the best fit. It is a quick and easy way to see the data and trend, similar to what you might do in a program like Tableau.

In [ ]:
g = sns.lmplot(data=film_class_df,
               x="Elongation at Break", y="Tensile Strength", 
               hue="FilmType")

### 7.2.5. Faceted histogram

In Unit 5, we explored the `Axes.hist()` function for the creation of histograms. In that unit, we plotted histograms on a single axis. When there are many categories to be explored, a single axis can get crowded and difficult to read. Using what we learned in section 7.1, we could create faceted plots by from scratch. The [`seaborn.displot()`](https://seaborn.pydata.org/generated/seaborn.displot.html) function will automatically create these facets on our behalf, speeding up the creation of new plots.

To create the grid, we supply column names to the `row` and `col` parameters in the function. This data sources contains only the tensile modulus measurements (for many films), so no filtering of the `DataFrame` is required.

In [ ]:
# read the data file
modulus_df = pd.read_csv('../../data/modulus.csv')

# faceted plot of histograms
# the facet_kws (keywords) parameter is used to set the labels on the margins
g = sns.displot(data=modulus_df,
                x='Measurement', bins=15, 
                row='Direction', col='FilmType',
                facet_kws=dict(margin_titles=True))

--------------
## Next Steps:

1. Complete the [Unit 7 Problems](./unit07-solutions.ipynb) to test your understanding
2. Advance to [Unit 8](../08-image-analysis/unit08-lesson.ipynb) when you're ready for the next step